In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

# Now Explicit ADR

In [2]:
def _AD_matrix(cfld, cfla, PHI, NX, Nsteps):
    T = np.zeros([NX,NX])
    T[0][0] = 1
    T[-1][-1] = 1
    
    a = cfld + cfla/2
    b = cfld - cfla/2
    c = 1-2*cfld

    for n in range(1,NX-1):
        T[n][n-1]=a
        T[n][n]=c
        T[n][n+1]=b
        
    f = np.zeros([Nsteps, NX])

    sizex = 1
    dx = 1. /NX
#     print(dx)
    x = np.linspace(0,1,NX)

    sigma = .1*sizex
    f[0][1:-1] = np.exp(-.5*np.power((x[1:-1]-sizex/2)/sigma,2))
    f[0,0] = PHI
    f[0,-1] = PHI
    
    return T,f


## Let's do this with linear chemistry at first and constant advection

In [3]:

def _ADR_matrix(cfld, cfla, kappa, PHI, NX, Nsteps):
    T = np.zeros([NX,NX])
    T[0][0] = 1
    T[-1][-1] = 1
    
    a = cfld + cfla/2
    b = cfld - cfla/2
    c = 1-2*cfld + kappa

    for n in range(1,NX-1):
        T[n][n-1]=a
        T[n][n]=c
        T[n][n+1]=b
        
    f = np.zeros([Nsteps, NX])

    sizex = 1
    dx = 1. /NX
    x = np.linspace(0,1,NX)

    sigma = .1*sizex
    f[0][1:-1] = np.exp(-.5*np.power((x[1:-1]-sizex/2)/sigma,2))
    f[0,0] = PHI
    f[0,-1] = PHI
    
    return T,f

def _ADR_matrix_PBC(cfld, cfla, kappa, PHI, NX, Nsteps):

    a = cfld + cfla/2
    b = cfld - cfla/2
    c = 1-2*cfld + kappa

    T = np.zeros([NX,NX])
    T[0][0] = c
    T[-1][-1] = c
    
    T[0][1] = b
    T[-1][-2] = a
    
    T[0][-1] = a
    T[-1][0] = b
    
    for n in range(1,NX-1):
        T[n][n-1]=a
        T[n][n]=c
        T[n][n+1]=b
        
    f = np.zeros([Nsteps, NX])

    sizex = 1
    dx = 1. /NX
    x = np.linspace(0,1,NX)

    sigma = .1*sizex
    f[0][:] = np.exp(-.5*np.power((x[:]-sizex/2)/sigma,2))

    print(T)
    return T,f
def ADR_with_history(cfld, cfla, kappa, PHI, NX, Nsteps):
    T,f = _ADR_matrix_PBC(cfld, cfla, kappa, PHI, NX, Nsteps)
    dx    = 1/float(NX-1)
    x = np.linspace(0,1,NX)

    for t in range(1,Nsteps):
        f[t] = np.dot(T,f[t-1])
    return x, f
def plot_10_percent(x,f):
    lw = 2
    
    stepsize = np.floor(f.shape[0]/10)
    plt.figure(figsize = (9,9))
    # plt.plot(x,f[0],'k--',alpha = 1, lw = lw+.5,label = 0)
    for step in np.arange(0,f.shape[0],stepsize):
        plt.plot(x,f[step],'k',alpha = .9*step/f.shape[0]+.1, lw = lw, label = np.int(step))

#     plt.ylim([0,1])
    plt.xlabel('x')
    plt.legend(fontsize = 14)
    plt.show()
# %time diffusion_fast(.4,1,100,34000)
def calc_H(f,dx):
    tot = 0
    for i in f.flatten():
        tot += i*i*dx
    return tot
def calc_M0(f,dx):
    tot =0
    for i in f.flatten():
        tot += i*dx
    return tot
def calc_M1(f,x,dx):
    return np.dot(f,x*dx)

def norm(x):
    np.array(x)
    x/=np.max(x)
    return x
def unit_H(f, dx, M0):
    tot = 0 
    for i in f.flatten():
        tot += i*i**dx/M0
    return tot
def diag(x,f):
    dx = x[1]-x[0]
    list_H = []
    list_M0 = []
    list_M1 = []
    list_unit_H = []
    for i in range(len(f)-1):
        list_H.append(calc_H(f[i],dx))
        list_M0.append(calc_M0(f[i],dx))
        list_M1.append(calc_M1(f[i],x,dx))
        list_unit_H.append(unit_H(f[i],dx,list_M0[-1]))
#     print(M0_prime)
    print(list_H[1:6])
    plt.figure(figsize =(9,9))
    plt.plot(list(range(len(f)-1)),norm(list_H),'k',lw=5 , label = r"$H_2$")
    plt.plot(list(range(len(f)-1)),norm(list_M0),':',lw=5 , label = r"$M_0$")
#     plt.plot(list(range(len(f)-1)),norm(list_M0),':',lw=5 , label = r"$M_0$")
    plt.plot(list(range(len(f)-1)),norm(list_unit_H),':',lw=5 , label = r"$H/M0$")

#     plt.plot(list(range(len(f)-2)),norm(M0_prime),label = 'M0_prime')
#     plt.plot(list(range(len(f)-1)),norm(list_M1),'--',lw=5 , label = r"$M_1$")
#     print(np.array(list_M1)-np.array(list_M0))
    print(norm(list_M1))
    fs = 20
    plt.ylabel("Normalized Value",fontsize =fs)
    plt.legend(fontsize = fs)

    plt.xlabel("Time step",fontsize = fs)
    plt.show()
    return list(range(len(f)-1)), list_M0, list_M1, list_H, list_unit_H
def init(Nsteps, NX,sizeX):
    sigma = .1*sizeX
    x = np.linspace(0,sizeX,NX)
    f = np.zeros([Nsteps, NX])
    f[0][:] = np.exp(-.5*np.power((x[:]-sizeX/2)/sigma,2))
    return x,f


In [4]:
dif = 0.5
vel = 0.9
sizeX = 1
Nsteps = 5000
NX = 100
dx = sizeX/(NX-1)
x = np.linspace(0,sizeX,NX)
sigma = .1*sizeX

f = np.zeros([Nsteps, NX])
f_AD = np.zeros([Nsteps, NX])
f[0][:] = np.exp(-.5*np.power((x[:]-sizeX/2)/sigma,2))
f_AD[0][:] = np.exp(-.5*np.power((x[:]-sizeX/2)/sigma,2))

logis_a = 10
logis_b = 7
dt_dif  = dx*dx/dif
dt_adv  = dx/vel 
if logis_a != 0:
    dt_chem = 1/logis_a
dt = .25*np.min([dt_dif, dt_adv, dt_chem])

alpha = vel*dt/dx
delta = dif*dt/(dx*dx)
# with logistic kappa will be constantly changing
#kappa = 
print(dt)

a = delta + .5*alpha
b = delta -.5*alpha
c = 1-a-b

T_base = np.zeros([NX,NX])
T_base[0][0] = c
T_base[-1][-1] = c

T_base[0][1] = b
T_base[-1][-2] = a

T_base[0][-1] = a
T_base[-1][0] = b

for n in range(1,NX-1):
    T_base[n][n-1]=a
    T_base[n][n]=c
    T_base[n][n+1]=b
    

chem = (logis_a-logis_b)*f[0]*dt
T = T_base +chem*np.identity(NX)
print(T)
for it in range(1,Nsteps):
    chem = (logis_a-logis_b*f[it-1])*dt
    T = T_base + chem*np.identity(NX)
    f[it] = np.dot(T,f[it-1])
for it in range(1,Nsteps):
    f_AD[it] = np.dot(T_base,f_AD[it-1])
plot_10_percent(x,f)
plot_10_percent(x,f_AD)
out = diag(x,f)
out = diag(x,f_AD)

5.10152025304e-05
[[ 0.5         0.24772727  0.         ...,  0.          0.          0.25227273]
 [ 0.25227273  0.5         0.24772727 ...,  0.          0.          0.        ]
 [ 0.          0.25227273  0.5        ...,  0.          0.          0.        ]
 ..., 
 [ 0.          0.          0.         ...,  0.5         0.24772727  0.        ]
 [ 0.          0.          0.         ...,  0.25227273  0.5         0.24772727]
 [ 0.24772727  0.          0.         ...,  0.          0.25227273  0.5       ]]


IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [ ]:
max_v = 825
plt.plot(out[0][:max_v],out[1][:max_v])


In [ ]:
a = delta + .5*alpha
b = delta -.5*alpha
c = 1-a-b

T_base = np.zeros([NX,NX])
T_base[0][0] = c
T_base[-1][-1] = c

T_base[0][1] = b
T_base[-1][-2] = a

T_base[0][-1] = a
T_base[-1][0] = b

for n in range(1,NX-1):
    T_base[n][n-1]=a
    T_base[n][n]=c
    T_base[n][n+1]=b
    

chem = (logis_a-logis_b)*f[0]*dt
T = T_base +chem*np.identity(NX)
for it in range(1,Nsteps):
    chem = (logis_a-logis_b)*f[it-1]*dt
    T = T_base + chem*np.identity(NX)
    f[it] = np.dot(T,f[it-1])
plot_10_percent(x,f)
out = diag(x,f)

In [ ]:
tot = 0
for i in range(NX):
    tot += f[0][i]*x[i]*dx
print(tot*4)

In [ ]:
plt.plot(out[0],np.array(out[3])*4)
# plt.plot(out[0][0:-1],out[2])
# plt.plot(out[0],out[-1])

In [ ]:
x,f = ADR_with_history(.2, .2, .005, 1,100, 500)
print(f.shape)
plot_10_percent(x,f)

# Does chem commute with AD

In [ ]:
a = delta + .5*alpha
b = delta -.5*alpha
c = 1-a-b

T_base = np.zeros([NX,NX])
T_base[0][0] = c
T_base[-1][-1] = c

T_base[0][1] = b
T_base[-1][-2] = a

T_base[0][-1] = a
T_base[-1][0] = b

for n in range(1,NX-1):
    T_base[n][n-1]=a
    T_base[n][n]=c
    T_base[n][n+1]=b
Nsteps = 5000
x,f_AD_R = init(Nsteps, NX, sizeX)
x,f_R_AD = init(Nsteps, NX, sizeX)
logis_a = 10
logis_b = 1

for it in range(1,Nsteps):
    chem = (logis_a-logis_b*f_AD_R[it-1])*dt
    T_chem = np.identity(NX) + chem*np.identity(NX)
    f_AD_R[it] = np.dot(T_base,f_AD_R[it-1])
    f_AD_R[it] = np.dot(T_chem,f_AD_R[it])
    
for it in range(1,Nsteps):
    chem = (logis_a-logis_b*f_R_AD[it-1])*dt
    T_chem = np.identity(NX) + chem*np.identity(NX)
    f_R_AD[it] = np.dot(T_chem,f_R_AD[it-1])
    f_R_AD[it] = np.dot(T_base,f_R_AD[it])

plot_10_percent(x,f_AD_R)
plot_10_percent(x,f_R_AD)
f_R_AD[-1]-f_AD_R[-1]

In [ ]:
abs_dif = np.sum(np.abs(f_R_AD-f_AD_R),axis=1)
rel_dif = np.sum(np.abs(f_R_AD-f_AD_R)/f_R_AD,axis=1)/f_R_AD.shape[1]
plt.figure(figsize = (9,9))
plt.plot(list(range(len(abs_dif))),rel_dif*100)
plt.ylabel("% Difference",fontsize = 16)
plt.xlabel("Time Step", fontsize = 16)
plt.show()

In [ ]:
print(abs_dif)

# Implicit Crank-Nicolson
With doing chemistry then AD as separate steps because we can separate the operators

In [ ]:
#start with just chemistry
h = dt
logis_a  = 10
logis_b = 1
Nsteps = 50000
x,f = init(Nsteps = Nsteps, NX = 100, sizeX=1)

def chem(f):
    return logis_a*f-logis_b*f*f
for it in range(Nsteps-2):
    intermediate = f[it] + chem(f[it])*h
    f[it+1] = f[it] + chem(f[it])*h/2+chem(intermediate)*h/2
plot_10_percent(x,f)
diag(x,f)

In [ ]:
# exponential integration

for it in range(1,Nsteps):
    for j in range(NX):
        

# Simple ODE + Adaptive time stepping

In [ ]:
def calc_ode(t, h, x0):
    N_steps = np.int(t/h)
    x = np.zeros([N_steps+1])
    t = np.linspace(0,1,N_steps+1)
    x[0] = x0
    for i in range(1,N_steps+1):
        x[i] = h*x[i-1]**2+x[i-1]
    return t,x
def x_t(t,x0):
    return x0/(1-x0*t)

plt.figure(figsize=(9,9))
t,x = calc_ode(1,.1,1)
plt.plot(t,x,label = "h = {:}".format(t[1]-t[0]))

t,x = calc_ode(1,.01,1)
plt.plot(t,x,label = "h = {:}".format(t[1]-t[0]))

t,x = calc_ode(1,.001,1)
plt.plot(t,x,label = "h = {:}".format(t[1]-t[0]))


t = np.linspace(0,.99,N_steps)
plt.plot(t,x_t(t,x0),label = "Analytic")
plt.ylim([0,100])
plt.xlim([.6,1])
plt.legend(fontsize = 16)

In [ ]:
def calc_h(h_cur, h_old, q, epsilon, y_cur, y_old):
    return q*(h_old-h_cur)*epsilon/np.abs(y_cur-y_abs)

In [ ]:
# adapative a la sauro


def calc_ode_adap(t, epsilon, x0,q):
    
    h = epsilon*x0*q
    x = np.zeros([N_steps+1])
    time_list = [0]
    n_steps = 0
    x = [x0]
    i = 0
    while time_list[-1] < 1:
        i += 1
        time_list.append(time_list[-1]+h)
        x.append(h*x[i-1]**2+x[i-1])
        if h < epsilon/x[i]:
            h = epsilon *q/ x[i]
    return time_list,x

In [ ]:
plt.figure(figsize=(9,9))
x0 = .8
eps = .1
t,x = calc_ode_adap(1,eps,x0,q=.9)
print(len(t))
plt.plot(t,x,label = r"Adaptive, $\epsilon = {:}$".format(eps))

eps = .01
t,x = calc_ode_adap(1,eps,x0,q=.9)
print(len(t))
plt.plot(t,x,label = r"Adaptive, $\epsilon = {:}$".format(eps))

eps = .001
t,x = calc_ode_adap(1,eps,x0,q=.9)
print(len(t))
plt.plot(t,x,label = r"Adaptive, $\epsilon = {:}$".format(eps))


t = np.linspace(0,1,N_steps)
plt.plot(t,x_t(t,x0),label = "Analytic")
# plt.ylim([0,100])
# plt.xlim([.6,1])
plt.ylabel("Value",fontsize = 16)
plt.xlabel("t",fontsize = 16)
plt.legend(fontsize = 16)

# ADR in 2D

In [ ]:
def ADR_2D_update(t,f,i,j):
    a = delta + alpha/2
    b = delta - alpha/2
    c = 1-4*cfld
    f[t,i,j]  = a*f[t-1,i-1,j]+b*f[t-1,i+1,j]
    f[t,i,j] += a*f[t-1,i,j-1]+b*f[t-1,i,j+1]
    f[t,i,j] += c*f[t-1,i,j]
    f[t,i,j] += (logis_a-logis_b*f[t-1][i][j])*f[t-1][i][j]

In [ ]:
from scipy.stats import multivariate_normal
from mpl_toolkits.mplot3d import Axes3D

logis_b = 1
Nsteps = 40
NX = 100
sizex  = 1.0 
dif    = 1.0 


dx    = sizex/float(NX-1)


x = np.linspace(0,1,NX)
f = np.zeros([Nsteps,NX,NX])
PHI = 1
D = 1
# h 
cfld = .2


#Parameters to set
mu_x = np.mean(x)
variance_x = .1

mu_y = np.mean(x)
variance_y = .1


X, Y = np.meshgrid(x,x)
pos = np.empty(X.shape + (2,))
pos[:, :, 0] = X; pos[:, :, 1] = Y
rv = multivariate_normal([mu_x, mu_y], [[variance_x, 0], [0, variance_y]])

f[0] +=rv.pdf(pos)/np.max(rv.pdf(pos))
f[0,0,:]  = PHI
f[0,-1,:] = PHI
f[0,:,0]  = PHI
f[0,:,-1] = PHI

# im=plt.imshow(data[None], aspect='auto',cmap=cmap, norm=norm)
# cbar = plt.colorbar(im)
# cbar
plt.imshow(f[0],vmin=0,vmax=1)
plt.colorbar()
plt.show()

def new_f_2D(Nsteps, NX, PHI):
    f = np.zeros([Nsteps,NX,NX])
    f[0]+=rv.pdf(pos)/np.max(rv.pdf(pos))*70
    f[0,0,:]  = PHI
    f[0,-1,:] = PHI
    f[0,:,0]  = PHI
    f[0,:,-1] = PHI
    return f
def loop_2D(f,PHI,Nsteps, NX):
    for t in range(1,Nsteps):
        #Dirichlet Boundary Conditions
        f[t-1,0,:]  = PHI
        f[t-1,-1,:] = PHI
        f[t-1,:,0]  = PHI
        f[t-1,:,-1] = PHI
        # Updating 
        for i in range(1,NX-1):
            for j in range(1,NX-1):
                ADR_2D_update(t,f,i,j)

In [ ]:

loop_2D(f,PHI, Nsteps, NX)
plt.imshow(f[0],vmin=0,vmax=1)
plt.colorbar()
plt.show()
plt.imshow(f[-2],vmin=0,vmax=1)
plt.colorbar()
plt.show()
diag(x,f)

In [ ]:
plt.imshow(f[0],vmin=0,vmax=1)
plt.colorbar()
plt.show()
plt.imshow(f[1],vmin=0,vmax=1)
plt.colorbar()
plt.show()
diag(x,f)